In [1]:
import os
import polars as pl
import utils.constants as constants
from utils.utils import read_gff_output, extract_subsequences
from Bio import SeqIO
import argparse

from logging_config import logger

def extract_array_flanks(df: pl.DataFrame,fasta_records: list,
                         flank_size: int =constants.FLANK_SIZE,flank_type: str="flanks") -> None:  


    grouped = df.group_by(["feature"])

    for group_key,group_df in grouped:
    
        #extract left flanks
        left_flanks = extract_subsequences(fasta_records, group_df,flanks="left",flank_size=flank_size)
        #extract right flanks
        right_flanks = extract_subsequences(fasta_records, group_df,flanks="right",flank_size=flank_size)
        #merge them together
        left_flanks.extend(right_flanks)
        
        #Write the subsequences to a new FASTA file
        output_file = constants.FLANKS_SAVE_ROOT + group_key[0] + "_"+ flank_type +".fasta"
        SeqIO.write(left_flanks, output_file, 'fasta')

def load_in_df_from_list_of_files(file_list: list,squish=False):
    
    df_list = []

    if squish:
        for i in file_list:
            name = i.split("/")[-1]
            name = name[:-4]
            name = name.split("_")[0:4]
            name = "_".join(name)
            tmp = pl.read_csv(i,dtypes={"index":int,
            "actual":float,
            "roll_mean":float},separator="\t")
            tmp = tmp.with_columns(
                name = pl.lit(name)
            )
            print(tmp)
            df_list.append(tmp)
    else: 
        for i in file_list:
            name = i.split("/")[-1]
            name = name[:-4]
            name = name.split("_")[0:3]
            name = "_".join(name)
            tmp = pl.read_csv(i,dtypes={"index":int,
            "actual":float,
            "roll_mean":float},separator="\t")
            tmp = tmp.with_columns(
                name = pl.lit(name)
            )
            df_list.append(tmp)
    return df_list










In [2]:
fasta_records = list(SeqIO.parse("../input.fasta", 'fasta'))
    

file_list = os.listdir("../results/kmer_analysis/" + "data")


file_list = ["../results/kmer_analysis/" + "data/" + string for string in file_list]

df_list = load_in_df_from_list_of_files(file_list=file_list,squish=constants.SQUISH)



shape: (1_461, 4)
┌───────┬────────┬───────────┬───────────────────────────────────┐
│ index ┆ actual ┆ roll_mean ┆ name                              │
│ ---   ┆ ---    ┆ ---       ┆ ---                               │
│ i64   ┆ f64    ┆ f64       ┆ str                               │
╞═══════╪════════╪═══════════╪═══════════════════════════════════╡
│ 0     ┆ 14.0   ┆ 10.238095 ┆ Marev4contig105_MelSat49-103_240… │
│ 1     ┆ 14.0   ┆ 10.095238 ┆ Marev4contig105_MelSat49-103_240… │
│ 2     ┆ 14.0   ┆ 9.952381  ┆ Marev4contig105_MelSat49-103_240… │
│ 3     ┆ 14.0   ┆ 9.809524  ┆ Marev4contig105_MelSat49-103_240… │
│ 4     ┆ 14.0   ┆ 9.619048  ┆ Marev4contig105_MelSat49-103_240… │
│ …     ┆ …      ┆ …         ┆ …                                 │
│ 1456  ┆ 0.0    ┆ 0.0       ┆ Marev4contig105_MelSat49-103_240… │
│ 1457  ┆ 0.0    ┆ 0.0       ┆ Marev4contig105_MelSat49-103_240… │
│ 1458  ┆ 0.0    ┆ 0.0       ┆ Marev4contig105_MelSat49-103_240… │
│ 1459  ┆ 0.0    ┆ 0.0       ┆ Marev4contig1

In [10]:

#find the min max values by unique array name in the filtered data
df_tot = pl.concat(df_list).filter(
    pl.col("roll_mean")<5.0
).group_by('name').agg(
    min_value=pl.col('index').min(),
    max_value=pl.col('index').max()
)


In [19]:
pl.concat(df_list).filter(
    pl.col("roll_mean")<5.0
).group_by('name').agg(
    min_value=pl.col('index').min(),
    max_value=pl.col('index').max()
)


name,min_value,max_value
str,i64,i64
"""Marev4contig3_…",479,4507
"""Marev4contig97…",421,4540
"""Marev4contig20…",350,3417
"""Marev4contig10…",452,4948
"""Marev4contig24…",454,4465
"""Marev4contig15…",224,3552
"""Marev4contig10…",480,1460
"""Marev4contig82…",0,2981
"""Marev4contig10…",267,4592


In [11]:

df_tot = df_tot.with_columns(
    query = pl.col("name").map_elements(lambda x: x.split("_")[1])
)
df_tot =df_tot.rename({
        "min_value" : "start",
        "max_value" : "end",
        "query" : "feature",
        "name" : "seqnames"
    })

df_tot = df_tot.with_columns(
    [
        (pl.lit("feature")).alias("group"),
        (pl.lit("*")).alias("strand"),
        (pl.lit("EuSatXplor")).alias("source"),
        (pl.lit(".")).alias("frame"),
        (pl.lit("100")).alias("score")
    ]
)



In [12]:
df_tot

seqnames,start,end,feature,group,strand,source,frame,score
str,i64,i64,str,str,str,str,str,str
"""Marev4contig16…",436,4948,"""MelSat52-97""","""feature""","""*""","""EuSatXplor""",""".""","""100"""
"""Marev4contig7_…",476,4466,"""MelSat44-96""","""feature""","""*""","""EuSatXplor""",""".""","""100"""
"""Marev4contig96…",437,1976,"""MelSat14-95""","""feature""","""*""","""EuSatXplor""",""".""","""100"""
"""Marev4contig60…",479,4505,"""MelSat67-85""","""feature""","""*""","""EuSatXplor""",""".""","""100"""
"""Marev4contig81…",69,3533,"""MelSat24-104""","""feature""","""*""","""EuSatXplor""",""".""","""100"""
"""Marev4contig45…",333,2103,"""MelSat18-88""","""feature""","""*""","""EuSatXplor""",""".""","""100"""
"""Marev4contig46…",444,1134,"""MelSat75-137""","""feature""","""*""","""EuSatXplor""",""".""","""100"""
"""Marev4contig24…",454,4465,"""MelSat78-97""","""feature""","""*""","""EuSatXplor""",""".""","""100"""
"""Marev4contig14…",456,4948,"""MelSat63-60""","""feature""","""*""","""EuSatXplor""",""".""","""100"""


In [9]:
df_tot = df_tot.select(
                ["seqnames","source","feature","start","end","score","strand","frame","group"]
    )



#read in the original arrays
df = read_gff_output("../results/data/tables/arrays.gff",headers=False)

df_real_edges = df_tot
if constants.SQUISH:
    df = df.with_columns(
        name = pl.col("seqnames")+"_" + pl.col("group") +"_" + pl.col("start").cast(pl.String)+"_" + pl.col("end").cast(pl.String)
    ).with_columns(
        len= pl.col("end") -pl.col("start")
    )


    df = df.join(df_real_edges,left_on="name",right_on="seqnames")

    df = df.with_columns(
        real_start=pl.col("start") - (500-pl.col("start_right"))).with_columns(
        real_end=pl.when(pl.col("len")<5000)
                    .then(pl.col("end_right") - pl.col("start_right") + pl.col("real_start")) 
                    .otherwise(pl.col("end") + (5000-pl.col("end_right")))
    ).select(
            ["seqnames","source","feature","real_start","real_end","score","strand","frame","group"]
    ).rename(
        {
            "real_start" : "start",
            "real_end" :"end"
        }
    )
    
    
    extract_array_flanks(df=df,fasta_records=fasta_records)

    extract_array_flanks(df=df,fasta_records=fasta_records,
                        flank_size=20,flank_type="microhomology")


seqnames,source,feature,start,end,score,strand,frame,group
str,str,str,i64,i64,i64,str,str,str
"""Marev4contig10…","""EuSatXplor""","""MelSat10-63""",3177183,3185444,100,"""-""",""".""","""MelSat10-63"""
"""Marev4contig3""","""EuSatXplor""","""MelSat21-55""",3821227,3836043,100,"""-""",""".""","""MelSat21-55"""
"""Marev4contig3""","""EuSatXplor""","""MelSat33-57""",3467635,3469779,100,"""+""",""".""","""MelSat33-57"""
"""Marev4contig13…","""EuSatXplor""","""MelSat74-97""",566370,576028,100,"""+""",""".""","""MelSat74-97"""
"""Marev4contig10…","""EuSatXplor""","""MelSat16-76""",3700660,3701889,100,"""+""",""".""","""MelSat16-76"""
"""Marev4contig11…","""EuSatXplor""","""MelSat38-86""",549829,550512,100,"""+""",""".""","""MelSat38-86"""
"""Marev4contig20…","""EuSatXplor""","""MelSat83-173""",35788,38855,100,"""-""",""".""","""MelSat83-173"""
"""Marev4contig17…","""EuSatXplor""","""MelSat51-80""",266028,283398,100,"""-""",""".""","""MelSat51-80"""
"""Marev4contig16…","""EuSatXplor""","""MelSat52-97""",378590,417366,100,"""+""",""".""","""MelSat52-97"""
